In [1]:
import requests
import pandas as pd
import matplotlib
import os
import math
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import numpy as np
from datetime import date, timedelta

In [2]:
# API Key to use Polygon.io
#
API_KEY = os.environ.get('POLYGON_API_KEY')

# URL Strings to Pull Data
#
TICKERS_URL = 'https://api.polygon.io/v3/reference/tickers?active=true&sort=ticker&order=asc&limit={}&apiKey={}'
FINANCIAL_URL = 'https://api.polygon.io/v2/reference/financials/{}?sort=calendarDate&order=desc&limit={}&type={}&apiKey={}'
DAILY_URL = 'https://api.polygon.io/v1/open-close/{}/{}?adjusted=true&apiKey={}'

In [3]:
# Get Ticker Data
#
limit = 1000

session = requests.Session()
r = session.get(TICKERS_URL.format(limit, API_KEY))
data = r.json()
tickers_df = pd.DataFrame(data['results'])
next_url = data['next_url']

# Loop Through Remaining Pages of Tickers
#
for i in range(0,11):
    
    r2 = session.get(next_url + '&apiKey=' + API_KEY)
    data2 = r2.json()
    tickers_df = pd.DataFrame.append(tickers_df,pd.DataFrame(data2['results']))
    next_url = data2['next_url']

tickers_df = tickers_df.reset_index(drop=True)

In [4]:
# Function Calls
#

# Get Market Price of Stock Yesterday
def GET_DAILY(TICKER_F):
    yesterday = date.today() - timedelta(days=1)
    session = requests.Session()
    r = session.get(DAILY_URL.format(TICKER_F,
                                     yesterday, API_KEY))
    data = r.json()
    close = data['close']
    return(close)

# Get Financial Data From Polygon.io
def GET_FINANCIALS(TICKER_F,LIMIT):
    r_F = session.get(FINANCIAL_URL.format(TICKER_F,LIMIT,'YA',API_KEY))
    data_F = r_F.json()
    fin_df = pd.DataFrame(data_F['results'])
    return(fin_df)

# Growth Rate
def calc_GR(P0, P1, YEARS):
    rate = (P1/P0)**(1/YEARS) - 1
    return rate

# Calculate the Linear Slope of Discrete Data
def calc_slope(X, Y):
    m = (len(Y)*sum(X*Y)-sum(X)*sum(Y))/(len(Y)*sum(X**2)-(sum(X))**2)
    return m.item()

# Average Calculator of Dataframe with NAN Values
def ave_calc(X):
    Y = X.to_numpy()
    n = len(Y)
    Y_SUM = 0
    SUM = 0
    for i in range(0,n-1):
        try:
            int(Y[i])
        except:
            SUM = SUM + 1
            continue
        Y_SUM = Y_SUM + Y[i]
        SUM = SUM + 1
    return Y_SUM/SUM

In [5]:
n_tickers = len(tickers_df) #get length of tickers dataframe

bigFive_df = pd.DataFrame() #create empty dataframe to add financial data to

# Loop Through Every Ticker and Pull Financial Data
for ii in range(0, n_tickers):

    # Set Ticker and Get Data
    #
    i_ticker = tickers_df.loc[ii, 'ticker']
    fin_df = GET_FINANCIALS(i_ticker, 1000)
    n = len(fin_df)

    # Set Years to Get ROIC and Calculate Growth Rates
    #
    ROIC_YEARS = np.array([10, 5, 3, 0])
    GR_YEARS = np.array([10, 5, 3, 1])
    k = len(GR_YEARS)
    
    # Get ROIC
    #
    try:
        #must exist for more than 10 years - if error the code will move on
        ROIC = fin_df[['calendarDate','returnOnInvestedCapital','priceToEarningsRatio','earningsPerBasicShare']]
        ROIC_arr = ROIC.iloc[n - ROIC_YEARS - 1]
        ROIC_df = ROIC_arr.reset_index(drop=True)
    except:
        continue
    
    string_df = pd.DataFrame(np.full((4,1),i_ticker),columns=['Tickers'])

    # Set Growth Metric Indexes
    #
    growthMetrics = ['Sales Growth','EPS Growth','Equity Growth','Free Cash Growth']
    growthIndexes = ['revenues','earningsPerBasicShare','bookValuePerShare','freeCashFlow']
    m = len(growthMetrics)
    
    try:
        GR = np.zeros((k,m))
        GR_m = np.zeros((m,1))
        for j in range(0, m):
            Index = growthIndexes[j]
            for i in range(0, k):
                years = GR_YEARS[i]
                p1 = fin_df.loc[n - 1, Index]
                p0 = fin_df.loc[n - years - 1, Index]
                GR[i,j] = calc_GR(p0, p1, years)
            GR_m[j,0] = calc_slope(GR_YEARS,GR[:,j])

        growthRates = pd.DataFrame(GR, columns=growthMetrics)
        
        # Calculate Current Market Value to Compare to Sticker Price
        equity_GR = ave_calc(growthRates['Equity Growth'])
        EPS_est = fin_df.loc[n-1,'earningsPerBasicShare']
        EPS_fut = EPS_est*(1+equity_GR)**10
        PE = fin_df.loc[n-1,'priceToEarningsRatio']
        Future_Market = EPS_fut*PE
        Market = Future_Market/(1.15)**10
        MOS_price = Market/2

        market_df = pd.DataFrame(np.full((4,1),MOS_price),columns=['Est Market Price'])
        
        sticker = GET_DAILY(i_ticker)
        sticker_df = pd.DataFrame(np.full((4,1),sticker),columns=['Sticker Price'])

        bigFive = pd.concat([ROIC_df, growthRates, string_df,market_df,sticker_df], axis=1)
        bigFive_df = pd.DataFrame.append(bigFive_df, bigFive)
    except:
        continue

today = date.today()

bigFive_df.to_csv('Data/mass_output_'+today.strftime('%Y-%m-%d')+'.csv') #write data to csv file

<ipython-input-4-8449c7cc3d54>:23: RuntimeWarning: invalid value encountered in double_scalars
  rate = (P1/P0)**(1/YEARS) - 1
<ipython-input-4-8449c7cc3d54>:23: RuntimeWarning: invalid value encountered in longlong_scalars
  rate = (P1/P0)**(1/YEARS) - 1
<ipython-input-4-8449c7cc3d54>:23: RuntimeWarning: divide by zero encountered in longlong_scalars
  rate = (P1/P0)**(1/YEARS) - 1
<ipython-input-4-8449c7cc3d54>:28: RuntimeWarning: invalid value encountered in double_scalars
  m = (len(Y)*sum(X*Y)-sum(X)*sum(Y))/(len(Y)*sum(X**2)-(sum(X))**2)
<ipython-input-4-8449c7cc3d54>:23: RuntimeWarning: divide by zero encountered in double_scalars
  rate = (P1/P0)**(1/YEARS) - 1
